<a href="https://colab.research.google.com/github/Kalit31/Data-Mining-Assignment/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install NRCLex

     |████████████████████████████████| 399kB 4.7MB/s 
  Created wheel for NRCLex: filename=NRCLex-3.0.0-cp37-none-any.whl size=43310 sha256=ce3853ea1fd7a564ff3dd6d14717f8379336601ea794e8cb6d687db9e377cb3e
  Stored in directory: /root/.cache/pip/wheels/17/31/64/035a8d245b4c217aeb8e8a2702d05dc91544b9c2334db72414
Successfully built NRCLex


In [ ]:
!pip install sentistrength

In [ ]:
!bash   #1.python -m textblob.download_corpora  # 2. exit

bash: cannot set terminal process group (58): Inappropriate ioctl for device
bash: no job control in this shell
/content# python -m textblob.download_corpora
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
/content# exit
exit


In [ ]:
# import necessary packages
import pandas as pd
import numpy as np

import seaborn as sns

from sentistrength import PySentiStr
from nrclex import NRCLex

import io

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import  LogisticRegression
from sklearn import svm

from scipy.sparse import coo_matrix, hstack

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

import warnings
warnings.filterwarnings("ignore")

In [ ]:
senti = PySentiStr()
senti.setSentiStrengthPath('/content/gdrive/My Drive/DM project/SentiStrengthCom.jar') # Note: Provide absolute path instead of relative path
senti.setSentiStrengthLanguageFolderPath('/content/gdrive/My Drive/DM project/SentStrength_Data')

# **Loading Dataset and Plots**

In [ ]:
# load all three datasets
trainFile = 'https://raw.githubusercontent.com/bansal01yash/DM-Proj/main/train2.tsv'
testFile = 'https://raw.githubusercontent.com/bansal01yash/DM-Proj/main/test2.tsv'
valFile = 'https://raw.githubusercontent.com/bansal01yash/DM-Proj/main/val2.tsv'

# add header to all three datasets
train_data = pd.read_csv(trainFile, delimiter='\t',  names=["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State", "Party",
                         "Barely True Cnt", "False Cnt", "Half True Cnt", "Mostly True Cnt", "Pants on Fire Cnt", "Context", "Justification"])
train_data.name = 'Training Data'

test_data = pd.read_csv(testFile, delimiter='\t',  names=["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State", "Party",
                         "Barely True Cnt", "False Cnt", "Half True Cnt", "Mostly True Cnt", "Pants on Fire Cnt", "Context", "Justification"])

test_data.name = 'Testing Data'

val_data = pd.read_csv(valFile, delimiter='\t', names=["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State", "Party",
                         "Barely True Cnt", "False Cnt", "Half True Cnt", "Mostly True Cnt", "Pants on Fire Cnt", "Context", "Justification"])
val_data.name = 'Validation Data'

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(10242, 15)
(1267, 15)
(1284, 15)


In [ ]:
train_data["train-test-val"] = 0
val_data["train-test-val"] = 1
test_data["train-test-val"] = 2

In [ ]:
df = pd.concat([train_data,test_data,val_data]).reset_index(drop=True)
df.shape

(12793, 16)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_multi = LabelEncoder()
true_labels = ['half-true', 'mostly-true', 'true']
text_features = ['Statement', 'Context', 'Subject', 'Job Title', 'Justification']
sentiment=['fear','anger', 'anticip', 'trust','surprise', 'positive', 'negative', 'sadness', 'disgust', 'joy']

def dataPreprocessing(df):

    # Removing Appropriate Null Values
    df = df[df['ID'].notna()]
    # df = df[df['Barely True Cnt'].notna()]
    # df = df[df['False Cnt'].notna()]
    # df = df[df['Mostly True Cnt'].notna()]
    # df = df[df['Pants on Fire Cnt'].notna()]
    # df = df[df['Half True Cnt'].notna()]
    df = df[df['Statement'].notna()]



    
    #for Binary Classification
    df['is_fake'] = df.Label.apply(lambda x: 1 if x in true_labels else 0)
    df.loc[:, 'Label'] = le_multi.fit_transform(df.Label)

    for feature in text_features:
        #df = df[df[feature].notna()]
        df[feature].fillna('None', inplace=True)
        df[feature] = df[feature].str.replace(r'[^\w\s]+', ' ')       # Punctuation deleting
        #df[feature] = df[feature].str.lower()                         # Lowercase 
        df[feature] = df[feature].str.replace('\w*\d+\w*','NUM')      # Numbers replacement with "NUM" token
        df[feature] = df[feature].str.replace('\s{2,}',' ')           # Extra whitespaces removing

    #text preprocessing for speaker
      #Less common speakers replacement with token "unknown_speaker" (Less common definition by 11th percentile)
      #One-hot-Encoding
    df['Speaker'].fillna('unkown_speaker', inplace=True)
    df.loc[~df.Speaker.isin(df.groupby('Speaker', as_index=False).count()[['Speaker', 'ID']].query("ID   > 11").Speaker),'Speaker'] = 'unknown_speaker'
    df = df.join(pd.get_dummies(df.Speaker, prefix='Speaker'))


    
    #text preprocessing for State
      #Deleting some noise
      #One-hot-Encoding
    df['State'].fillna('noStateInfo', inplace=True)
    df.loc[df.State.isin(['None', 'Unknown']), 'State'] = 'noStateInfo'
    df.loc[df.State.isin(['Tennesse']), 'State'] = 'Tennessee'
    df.loc[df.State.isin(['PA - Pennsylvania']), 'State'] = 'Pennsylvania'
    df.loc[df.State.isin(['Rhode island']), 'State'] = 'Rhode Island'
    df.loc[df.State.isin(['Tex']), 'State'] = 'Texas'
    df.loc[df.State.isin(['Virgiia','Virgina', 'Virginia director, Coalition to Stop Gun Violence']), 'State'] = 'Virginia'
    df.loc[df.State.isin(['Washington D.C.','Washington DC','Washington State', 'Washington, D.C.',]), 'State'] = 'Washington'

    df = df.join(pd.get_dummies(df.State, prefix='State'))

    #One-hot-Encoding for party
    df['Party'].fillna('none', inplace=True)
    df = df.join(pd.get_dummies(df.Party, prefix='Party'))

    sdf=pd.DataFrame(senti.getSentiment(df['Statement'],score='trinary'),columns=['s_negative','s_neutral','s_positive'])
    df.index=sdf.index
    df=pd.concat([df,sdf],axis=1)

    for y in sentiment:
        df[y] = 0

    for i in range (len(df)): 
        cur=(NRCLex(df['Statement'][i]).affect_frequencies)
        for y in sentiment:
            if y in cur:
                df[y][i] = 1


    return df

In [ ]:
df_pre=dataPreprocessing(df)

In [ ]:
traindata=df_pre[df_pre['train-test-val']==0]
print(traindata.shape)
valdata=df_pre[df_pre['train-test-val']==1]
print(valdata.shape)
testdata=df_pre[df_pre['train-test-val']==2]
print(testdata.shape)
{l: i for i, l in enumerate(le_multi.classes_)}

(10240, 289)
(1284, 289)
(1267, 289)


{'barely-true': 0,
 'false': 1,
 'half-true': 2,
 'mostly-true': 3,
 'pants-fire': 4,
 'true': 5}

In [ ]:
df_pre.to_csv('data_preprocessed.csv', index=False)